# Creating a Functional Conversational Chatbot
* **Created by:** Eric Martinez
* **For:** Software Engineering 2
* **At:** University of Texas Rio-Grande Valley

## Tutorial: A Basic Conversational Chatbot with LLM (has limitations)

#### Installing Dependencies

In [8]:
!pip -q install --upgrade gradio
!pip -q install --upgrade openai

#### Creating a basic Chatbot UI using Gradio

In [9]:
import gradio as gr
import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

def chat(message, history):
    history = history or []
    ai_reply = get_ai_reply(message)       
    history.append((message, ai_reply))
    return None, history, history
    
with gr.Blocks() as demo:
    with gr.Tab("Conversation"):
        with gr.Row():
            with gr.Column():
                chatbot = gr.Chatbot(label="Conversation")
                message = gr.Textbox(label="Message")
                history_state = gr.State()
                btn = gr.Button(value ="Send")
            btn.click(chat, inputs = [message, history_state], outputs = [message, chatbot, history_state])
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7875
Running on public URL: https://8b8aa5ec7c7f25f014.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


#### Limitations
* Hardcoded to 'gpt-3.5-turbo' in the UI
* No error-handling on the API request
* While the OpenAI function takes message history, the UI doesn't pass it through
* Doesn't use or allow prompt or 'system' message customization

## Tutorial: Improved Chatbot

The following snippet adds conversation history to the Gradio chat functionality, handles errors, and passes along the system message.

In [27]:
# Define a function to handle the chat interaction with the AI model
def chat(model, system_message, message, chatbot_messages, history_state):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        ai_reply = get_ai_reply(message, model=model, system_message=system_message, message_history=history_state)
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
    
    # Append the user's message and the AI's reply to the chatbot_messages list
    chatbot_messages.append((message, ai_reply))
    
    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "user", "content": message})
    history_state.append({"role": "assistant", "content": ai_reply})
    
    # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    return None, chatbot_messages, history_state

The following snippet adjusts the Gradio interface to include examples (included in a separate file in this repo), model selection, prompts or 'system' messages, storing conversation history.

In [28]:
import examples as chatbot_examples

# Define a function to return a chatbot app using Gradio
def get_chatbot_app(additional_examples=[], share=False):
    # Load chatbot examples and merge with any additional examples provided
    examples = chatbot_examples.load_examples(additional=additional_examples)
    
    # Define a function to get the names of the examples
    def get_examples():
        return [example["name"] for example in examples]

    # Define a function to choose an example based on the index
    def choose_example(index):
        system_message = examples[index]["system_message"].strip()
        user_message = examples[index]["message"].strip()
        return system_message, user_message, [], []

    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a dropdown to select examples
                    example_dropdown = gr.Dropdown(get_examples(), label="Examples", type="index")
                    # Create a button to load the selected example
                    example_load_btn = gr.Button(value="Load")
                    # Create a textbox for the system message (prompt)
                    system_message = gr.Textbox(label="System Message (Prompt)", value="You are a helpful assistant.")
                with gr.Column():
                    # Create a dropdown to select the AI model
                    model_selector = gr.Dropdown(
                        ["gpt-3.5-turbo", "gpt-4"],
                        label="Model",
                        value="gpt-3.5-turbo"
                    )
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")

                # Connect the example load button to the choose_example function
                example_load_btn.click(choose_example, inputs=[example_dropdown], outputs=[system_message, message, chatbot, history_state])
                # Connect the send button to the chat function
                btn.click(chat, inputs=[model_selector, system_message, message, chatbot, history_state], outputs=[message, chatbot, history_state])
        # Return the app
        return app

In [29]:
# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
get_chatbot_app().launch()

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.
